In [2]:
import sqlite3
import pandas as pd
# Conectando ao banco de dados
conn = sqlite3.connect("sistema_escolar.db")

In [5]:
#Mostrar as tabelas disponiveis
consulta_tabelas = pd.read_sql_query(
    "Select name FROM sqlite_master WHERE type= 'table';",conn)
consulta_tabelas

,name
0,alunos
1,professores
2,cursos
3,matriculas
4,notas


In [6]:
#Visualizar os dados da tabela alunos
pd.read_sql_query("SELECT * from alunos", conn)

,id,nome,data_nascimento,cidade
0,1,Ana Souza,2000-05-10,São Paulo
1,2,Carlos Silva,1999-08-22,Rio de Janeiro
2,3,Juliana Lima,2001-03-14,Belo Horizonte
3,4,Marcos Pereira,2000-12-02,Salvador


In [9]:
#Todos os cursos com carga Horari superior a 45h
pd.read_sql_query("""SELECT * From cursos WHERE carga_horaria > 45 """, conn)

,id,nome,carga_horaria
0,1,Matemática Básica,60
1,3,História do Brasil,50


In [7]:
#Todos os alunos da cidade de São Paulo
pd.read_sql_query("""SELECT nome, cidade From alunos where cidade = 'São Paulo' """, conn)

,nome,cidade
0,Ana Souza,São Paulo


In [10]:
#Vizualiar as notas dos alunos do curso de Matematica Basica
pd.read_sql_query(""" SELECT * From notas WHERE curso_id = 1""", conn)

,id,aluno_id,curso_id,nota,semestre
0,1,1,1,8.5,2023-1
1,3,2,1,9.0,2023-1


In [11]:
#Quantos alunos estão matriculados em cada curso
pd.read_sql_query("""Select Curso_id, COUNT(*) as total_matriculas FROM matriculas GROUP BY curso_id ORDER BY total_matriculas DESC""", conn)

,curso_id,total_matriculas
0,2,2
1,1,2
2,3,1


In [12]:
#Ver nome do aluno e curso que ele está matriculado
pd.read_sql_query("""SELECT a.nome AS aluno, c.nome AS curso, m.data_matricula FROM matriculas m JOIN alunos a ON m.aluno_id = a.id JOIN cursos c ON m.curso_id = c.id ORDER BY aluno""", conn)

,aluno,curso,data_matricula
0,Ana Souza,Matemática Básica,2023-02-01
1,Ana Souza,Gramática Avançada,2023-02-01
2,Carlos Silva,Matemática Básica,2023-02-01
3,Juliana Lima,História do Brasil,2023-02-01
4,Marcos Pereira,Gramática Avançada,2023-02-01


In [15]:
#Ver o nome do aluno, curso e nota tirada
pd.read_sql_query("""SELECT a.nome AS aluno, c.nome AS curso, n.nota, n.semestre FROM notas n JOIN alunos a ON n.aluno_id = a.id JOIN cursos c ON n.curso_id = c.id ORDER BY aluno""", conn)

,aluno,curso,nota,semestre
0,Ana Souza,Matemática Básica,8.5,2023-1
1,Ana Souza,Gramática Avançada,7.0,2023-1
2,Carlos Silva,Matemática Básica,9.0,2023-1
3,Juliana Lima,História do Brasil,6.5,2023-1
4,Marcos Pereira,Gramática Avançada,8.0,2023-1


In [16]:
#Media de nota por aluno
pd.read_sql_query("""SELECT c.nome AS curso, ROUND(AVG(n.nota), 2) AS media_nota FROM notas n JOIN cursos c ON n.curso_id = c.id GROUP BY c.nome ORDER BY media_nota DESC""", conn)

,curso,media_nota
0,Matemática Básica,8.75
1,Gramática Avançada,7.50
2,História do Brasil,6.50


In [19]:
#Alunos por cidade
pd.read_sql_query("""SELECT cidade, COUNT(*) AS total_alunos FROM alunos GROUP BY cidade ORDER BY total_alunos DESC""", conn)

,cidade,total_alunos
0,São Paulo,1
1,Salvador,1
2,Rio de Janeiro,1
3,Belo Horizonte,1


In [20]:
#Alunos acima da média geral
pd.read_sql_query("""SELECT a.nome, n.nota FROM notas n JOIN alunos a ON n.aluno_id = a.id WHERE n.nota > (SELECT AVG(nota) FROM notas) ORDER BY n.nota DESC""", conn)

,nome,nota
0,Carlos Silva,9.0
1,Ana Souza,8.5
2,Marcos Pereira,8.0


In [21]:
#Alunos com maiores notas no curso de matematica
pd.read_sql_query("""Select a.nome, n.nota FROM notas n JOIN alunos a ON n.aluno_id = a.id WHERE n.curso_id = 1 ORDER BY n.nota DESC LIMIT 3""", conn)

,nome,nota
0,Carlos Silva,9.0
1,Ana Souza,8.5


In [22]:
#Usando uma CTE para listar a média de nota por curso e filtrar os cursos com média > 8
pd.read_sql_query(""" WITH media_por_curso AS (SELECT curso_id, AVG(nota) AS media FROM notas GROUP BY curso_id)
SELECT c.nome AS curso, ROUND(m.media, 2) AS media_geral FROM media_por_curso m JOIN cursos c ON c.id = m.curso_id WHERE m.media > 8""", conn)

,curso,media_geral
0,Matemática Básica,8.75


In [23]:
#Alunos com mais de uma matricula (Em diferentes cursos)
pd.read_sql_query("""SELECT a.nome, COUNT(*) AS total_matriculas FROM matriculas m JOIN alunos a ON a.id = m.aluno_id GROUP BY a.nome HAVING COUNT(*) > 1""", conn)

,nome,total_matriculas
0,Ana Souza,2
